In [17]:
from datetime import datetime
import pandas


# Prepare

In [142]:
def get_employee_data(source_path:str) -> pandas.DataFrame:
  schema = {
    'Emp-id': int,
    'Emp-name': str,
    'Emp-lastname': str
  }
  employee_df = pandas.read_csv(filepath_or_buffer=source_path, dtype=schema)
  return employee_df

In [143]:
def get_team_data(source_path:str) -> pandas.DataFrame:
  schema = {
    'name': str,
    'type': str,
    'desc': str
  }
  team_df = pandas.read_csv(filepath_or_buffer=source_path, dtype=schema)

  return team_df

In [144]:
def get_team_history_data(source_path:str) -> pandas.DataFrame:

  schema = {
    'Start-date': str,
    'End-date': str,
    'type': str,
    'team': str,
    'Emp-id': int
  }
  date_columns = ['Start-date', 'End-date']
  team_history_df = pandas.read_csv(filepath_or_buffer=source_path, dtype=schema)

  for date_column in date_columns:
    team_history_df[date_column] = pandas.to_datetime(team_history_df[date_column], format='%Y-%m-%d')

  return team_history_df

In [161]:
def get_calendar_df(start_date:str, end_date:str) -> pandas.DataFrame:
  start_date = datetime.strptime(start_date, "%Y-%m-%d")
  end_date = datetime.strptime(end_date, "%Y-%m-%d")
  date_range = pandas.date_range(start_date, end_date)
  date_df = pandas.DataFrame({'date': date_range})

  return date_df

In [157]:
def fill_na_history_df(history_df: pandas.DataFrame) -> pandas.DataFrame:
  cleaned_history_df = history_df.sort_values(by=['Emp-id', 'Start-date'])
  cleaned_history_df['End-date'] = cleaned_history_df['End-date'].fillna(cleaned_history_df['Start-date'].shift(-1) + pandas.Timedelta(days=-1))
  return cleaned_history_df

In [167]:
def join_employee_team_history_df(employee_df: pandas.DataFrame, team_df: pandas.DataFrame, team_history_df: pandas.DataFrame) -> pandas.DataFrame:

  detail_team_history_df = team_history_df.merge(team_df, left_on="team", right_on="name")
  employee_team_history = detail_team_history_df.merge(employee_df, left_on="Emp-id", right_on="Emp-id")

  return employee_team_history


In [168]:
def cross_join_date(df: pandas.DataFrame, date_df: pandas.DataFrame) -> pandas.DataFrame:
  df['key'] = 0
  date_df['key'] = 0
  cross_date_df = date_df.merge(df, how='outer', on='key').drop(columns='key', axis=1)
  return cross_date_df

In [169]:
def filter_date_between_start_end(df: pandas.DataFrame):
  filtered_df = df[(df['date'] >= df['Start-date']) & (df['date'] <= df['End-date'])]
  return filtered_df

In [170]:
def display_result_df(df: pandas.DataFrame):
  display_columns = ['team', 'Emp-id', 'type', 'desc', 'Emp-name', 'Emp-lastname', 'date', 'Start-date', 'End-date']

  result_df = df[display_columns].sort_values(by=['date', 'Emp-id'])
  return result_df

In [140]:
sorted_employee_team_history['key'] = 0
date_df['key'] = 0
cross_date_df = date_df.merge(sorted_employee_team_history, how='outer', on='key').drop(columns='key', axis=1)

filtered_df = cross_date_df

filtered_df = filtered_df[(filtered_df['date'] >= filtered_df['Start-date']) & (filtered_df['date'] <= filtered_df['End-date'])]
display_columns = ['team', 'Emp-id', 'type', 'desc', 'Emp-name', 'Emp-lastname', 'date', 'Start-date', 'End-date']

result_df = filtered_df[display_columns].sort_values(by=['date', 'Emp-id'])
# print(cross_date_df)
print(result_df)



       team  Emp-id  type     desc Emp-name Emp-lastname       date  \
0      TU-1       1    TU     ซ่อม        a          aaa 2020-01-01   
4      TU-1       2    TU     ซ่อม        b          bbb 2020-01-01   
8      TU-2       3    TU     ซ่อม        c          ccc 2020-01-01   
12     TU-2       4    TU     ซ่อม        d          ddd 2020-01-01   
16     DW-1       5    DW  ติดตั้ง        e          eee 2020-01-01   
..      ...     ...   ...      ...      ...          ...        ...   
463    DW-1       8    DW  ติดตั้ง        h          hhh 2020-01-10   
467    DW-2       9    DW  ติดตั้ง        I          iii 2020-01-10   
471    DW-1      10    DW  ติดตั้ง        j          jjj 2020-01-10   
475  SALE-2      11  SALE      ขาย        k          kkk 2020-01-10   
479  SALE-2      12  SALE      ขาย        l          lll 2020-01-10   

    Start-date   End-date  
0   2020-01-01 2020-01-02  
4   2020-01-01 2020-01-02  
8   2020-01-01 2020-01-02  
12  2020-01-01 2020-01-02  
16  202

# Execution

In [173]:
source_employee_file_path = "../dataset/emp.csv"
source_team_file_path = "../dataset/team.csv"
source_team_history_file_path = "../dataset/team-history.csv"
target_file_location = "../dataset/result-employee-team-by-date.csv"
start_date = "2020-01-01"
end_date = "2020-01-10"


# read source data
employee_df = get_employee_data(source_path=source_employee_file_path)
team_df = get_team_data(source_path=source_team_file_path)
team_history_df = get_team_history_data(source_path=source_team_history_file_path)
date_df = get_calendar_df(start_date=start_date, end_date=end_date)

# cleaning
cleaned_team_history_df = fill_na_history_df(history_df=team_history_df)

# transform
joined_df = join_employee_team_history_df(employee_df=employee_df, team_df=team_df, team_history_df=cleaned_team_history_df)
cross_join_date_df = cross_join_date(df=joined_df, date_df=date_df)
filtered_df = filter_date_between_start_end(df=cross_date_df)

# display
result_df = display_result_df(df=filtered_df)

# save result
result_df.to_csv(target_file_location, header=True)